# EMA Crossovers 📈


## 1. Setup

In [ ]:
import os,sys
import duckdb
from pathlib import Path
import pandas as pd
import json

In [2]:
PROJECT_ROOT = Path.cwd().parents[0]

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"Project Root: {PROJECT_ROOT}")

Project Root: /Users/luyanda/workspace/QuantTrade


In [3]:
from utils.charts import render_lightweight_chart
from utils.duck import to_bt_daily_duckdb, to_bt_minute_duckdb
from utils.features import add_mas_duckdb, add_rsi_duckdb, add_emas_duckdb, ema_crossover_signals_duckdb

In [4]:
DB_MINUTE = PROJECT_ROOT / "data" / "processed" / "alpaca" / "price_minute_alpaca.duckdb"
print(f"DB_MINUTE: {DB_MINUTE}")
con_minute = duckdb.connect(str(DB_MINUTE), read_only=True)
tables = [t[0] for t in con_minute.execute("SHOW TABLES").fetchall()]
print("📋 Tables:", tables)


DB_MINUTE: /Users/luyanda/workspace/QuantTrade/data/processed/alpaca/price_minute_alpaca.duckdb
📋 Tables: ['alpaca_minute']


In [5]:
ETFS = ["SPY", "QQQ"]
CHARTS_DIR = PROJECT_ROOT / "charts" / "002_EMA_Crossovers"
IND_MA_WINDOWS = [20, 50, 200]
IND_EMA_WINDOWS = [12,26,50,200]
IND_RSI_PERIOD = 14
IND_RSI_BOUNDS = (30,70)

## 2. Helpers

## 3. Data Ingestion

In [6]:
# --- Ingest latest minute-level data ---
minute_data = {sym: to_bt_minute_duckdb(con_minute, "alpaca_minute", sym) for sym in ETFS}

for symbol in ETFS:
    print(minute_data[symbol].tail(1))

                       open    high     low   close  volume  trade_count  \
datetime                                                                   
2025-08-13 14:36:00  644.34  644.34  644.34  644.34   100.0          1.0   

                       vwap  
datetime                     
2025-08-13 14:36:00  644.34  
                       open    high     low   close  volume  trade_count  \
datetime                                                                   
2025-08-13 14:55:00  582.45  582.45  582.45  582.45   340.0          2.0   

                       vwap  
datetime                     
2025-08-13 14:55:00  582.45  


## 4. Data Quality Checks

U.S. Market (SPY, QQQ)
Assuming regular NYSE/Nasdaq trading hours:

| **Session**     | **Hours (ET)**   | **Duration** |
| --------------- | ---------------- | ------------ |
| Regular session | 09:30 – 16:00 ET | 6.5 hours    |
|                 |                  | 390 minutes  |

Expect around 390 rows per ETF

In [7]:
for symbol in ETFS:
    df = minute_data[symbol]
    print(f"\n🔍 {symbol}")
    print(f"  • Rows: {len(df)}")
    print(f"  • Date Range: {df.index.min().date()} → {df.index.max().date()}")
    print(f"  • Timezone-aware: {df.index.tz is not None}")
    # print(f"  • Missing 'close': {df['close'].isna().sum()}")

    # --- Drop timezone if needed ---
    df = df.copy()
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)

    # --- Identify all available intraday dates ---
    df["date"] = df.index.normalize()
    available_dates = df["date"].unique()

    # --- Construct full expected range (business days) ---
    expected_dates = pd.date_range(
        start=df.index.min().normalize(),
        end=df.index.max().normalize(),
        freq='B'
    )

    # --- Missing trading days entirely ---
    missing_dates = sorted(set(expected_dates) - set(available_dates))
    print(f"  • Missing Intraday Dates: {len(missing_dates)}")
    # if missing_dates:
    #     print("    Example:", missing_dates[:5])

    # --- Check for partial trading days (fewer than 390 rows) ---
    counts = df.groupby("date").size()
    partial_days = counts[counts < 390]
    print(f"  • Partial Intraday Days (<390 rows): {len(partial_days)}")
    # if not partial_days.empty:
    #     print("    Example:", partial_days.head())



🔍 SPY
  • Rows: 192320
  • Date Range: 2023-08-09 → 2025-08-13
  • Timezone-aware: False
  • Missing Intraday Dates: 22
  • Partial Intraday Days (<390 rows): 279

🔍 QQQ
  • Rows: 187087
  • Date Range: 2023-08-09 → 2025-08-13
  • Timezone-aware: False
  • Missing Intraday Dates: 22
  • Partial Intraday Days (<390 rows): 306


## 5. Feature Engineering

In [9]:
minute_data_ema = add_emas_duckdb(minute_data, con_minute, IND_EMA_WINDOWS)

In [10]:
minute_data_ema_cross = ema_crossover_signals_duckdb(minute_data_ema, con_minute, fast=12, slow=26)

In [14]:
render_lightweight_chart(
    minute_data_ema_cross["SPY"],
    symbol="SPY",
    out_html=CHARTS_DIR/"etf_cross_SPY.html",
    theme="dark",
    # ma_windows=IND_MA_WINDOWS,  
    ema_windows=[12, 26],          
    # rsi_period=IND_RSI_PERIOD,             
    # rsi_bounds=IND_RSI_BOUNDS,
    timeframes=["1d", "1h", "15m"],
    default_tf="15m",
    watermark_text="SPY — {tf}",
    watermark_opacity=0.0001,
    assets_rel="../../utils/static"
)

PosixPath('/Users/luyanda/workspace/QuantTrade/charts/002_EMA_Crossovers/etf_cross_SPY.html')

In [13]:
con_minute.close()